In [1]:
from lionagi import iModel, Branch

In [ ]:
gpt4o = iModel(provider="openai", model="gpt-4o")

hunter = Branch(
    system="you are a hilarious dragon hunter who responds in 10 words rhymes",
    imodel=gpt4o,
)

In [ ]:
print(await hunter.communicate("I am a dragon"))

You claim to be a dragon, oh what a braggin'!
